In [2]:
import numpy as np
import pandas as pd

In [6]:
# read NFA from csv
inputFile= "C:/Users/Admin/Desktop/FA CSV input files/NFA.csv"
NFA = pd.read_csv(inputFile, index_col="states",dtype='str') 


# convert data to appropriate type (using frozen set for states as they are hashable and can be used as index)
NFA.isInitial = NFA["isInitial"].map({'FALSE':False, 'TRUE':True}) # bool
#NFA.index = [frozenset(x) for x in NFA.index] # bool
NFA.isFinal = NFA["isFinal"].map({'FALSE':False, 'TRUE':True}) # bool
# convert string transitions to sets.
for s in [x for x in list(NFA.columns) if x.find("input")!=-1]: 
    NFA[s]=[frozenset(x.split(",")) if x==x else frozenset() for x in NFA[s]]


print("Given NFA\n",NFA)
q0= NFA.loc[NFA["isInitial"]==True].index.values[0]
print("inital Stated of NFA=",q0)
finalStates=set(NFA.loc[NFA["isFinal"]==True].index.values.tolist())
print("final states: ", finalStates)
symbols= [x for x in list(NFA.columns) if x.find("input")!=-1]
print("symbols: ", symbols)



Given NFA
        input_0 input_1  isInitial  isFinal
states                                    
A       (B, A)     (A)       True    False
B           ()     (C)      False    False
C          (D)      ()      False    False
D           ()     (D)      False     True
inital Stated of NFA= A
final states:  {'D'}
symbols:  ['input_0', 'input_1']


In [4]:
# initialize dataframe for DFA
# add initial state of DFA (=q0 of NFA)
# keeping a marker  "added" in DFA to check transitions for which states have been 
DFA= pd.DataFrame(index= [frozenset([q0])],columns= list(NFA.columns)+["added"])
DFA.loc[frozenset([q0]), "isInitial"]= True

# for all states, fill transitions. If new states is reached add it in DFA table
while (DFA[DFA['added'].isnull()].shape[0]>0):
    
    currentstate= DFA[DFA['added'].isnull()].head(1).index.values[0] # get 1 state from table for which tran as not defined yet
    
    if len(set.intersection(set(currentstate),set(finalStates)))>0 : # check if this state can be final state
        DFA.loc[currentstate, "isFinal"]= True
        
    
    for s in symbols: # for all inputs define the transition of current state 
        desS= frozenset.union(* [ NFA.loc[x,s] for x in currentstate] )       
        DFA.loc[currentstate,s]=desS        
        
        if desS not in DFA.index.values: # if a new state is reached add it to the DFA table
            DFA= DFA.append(pd.DataFrame(index= [frozenset(desS)],columns= list(DFA.columns)))
            
    DFA.loc[currentstate, "added" ]= True # toggle the marker to show that the trans have been defined for this state 
    
DFA = DFA.drop('added', 1)   # discarding the marker column
    

In [5]:
print("Given NFA \n",NFA)
print("\n\n Corresponding DFA \n", DFA)

Given NFA 
        input_0 input_1  isInitial  isFinal
states                                    
A       (B, A)     (A)       True    False
B           ()     (C)      False    False
C          (D)      ()      False    False
D           ()     (D)      False     True


 Corresponding DFA 
              input_0    input_1 isInitial isFinal
(A)           (B, A)        (A)      True     NaN
(B, A)        (B, A)     (A, C)       NaN     NaN
(A, C)     (D, B, A)        (A)       NaN     NaN
(D, B, A)     (B, A)  (D, A, C)       NaN    True
(D, A, C)  (D, B, A)     (D, A)       NaN    True
(D, A)        (B, A)     (D, A)       NaN    True
